In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import scipy.fft
import os

In [14]:
def read_u32_le(f, off):
    f.seek(off)
    return int.from_bytes(f.read(4), 'little', signed=False)

def read_i32_le(f, off):
    f.seek(off)
    return int.from_bytes(f.read(4), 'little', signed=True)

def read_u16_le(f, off):
    f.seek(off)
    return int.from_bytes(f.read(2), 'little', signed=False)

def parse_params(fp):
    noSamples  = read_u32_le(fp, OFF_NO_SAMPLES)
    noViews    = read_u32_le(fp, OFF_NO_VIEWS)
    noViewsSec = read_u32_le(fp, OFF_NO_VIEWSSEC)
    noSlices   = read_u32_le(fp, OFF_NO_SLICES)
    dt_code    = read_u16_le(fp, OFF_DATATYPE)
    noEchoes   = read_u32_le(fp, OFF_NO_ECHOES)
    noExps     = read_u32_le(fp, OFF_NO_EXPS)
    return dict(
        noSamples=noSamples, noViews=noViews, noViewsSec=noViewsSec,
        noSlices=noSlices, noEchoes=noEchoes, noExperiments=noExps,
        dataTypeCode=dt_code
    )

def sign_extend_24(u):
    """u: np.uint32 array of low-24-bit values"""
    v = (u & 0x00FFFFFF).astype(np.int32)
    neg = v & 0x00800000
    v[neg != 0] -= 0x01000000
    return v

def read_raw_firtech(path: Path):
    with path.open('rb') as f:
        params = parse_params(f)
        f.seek(0, 2)
        file_size = f.tell()

        # 计算总样本数
        P = params
        dims = (P["noExperiments"], P["noEchoes"], P["noSlices"],
                P["noViewsSec"], P["noViews"], P["noSamples"])
        total_points = np.prod(dims, dtype=np.int64)

        dt = P["dataTypeCode"]
        if dt in (0x00, 0x02):  # complex int (24-bit stored in 4B slots)
            bytes_per_point = 8  # I(4B) + Q(4B)
            count_u32 = total_points * 2  # I & Q
            with path.open('rb') as f2:
                f2.seek(DATA_START)
                iq_u32 = np.fromfile(f2, dtype='<u4', count=count_u32)
            if iq_u32.size != count_u32:
                raise ValueError("File too short for declared dimensions. "
                                 f"Expected {count_u32} 32-bit words, got {iq_u32.size}.")
            # 取低24位并做符号扩展
            iq24 = sign_extend_24(iq_u32)
            I = iq24[0::2]
            Q = iq24[1::2]
            data = I.astype(np.float32) + 1j * Q.astype(np.float32)
        elif dt == 0x01:  # ADC 16-bit (single channel stream)
            bytes_per_point = 2  # 仅一路，非I/Q
            with path.open('rb') as f2:
                f2.seek(DATA_START)
                data = np.fromfile(f2, dtype='<i2', count=total_points)
            if data.size != total_points:
                raise ValueError("File too short for declared dimensions (ADC).")
            data = data.astype(np.float32)  # 可按需保留为int16
        else:
            raise ValueError(f"Unknown DataTypeCode: 0x{dt:02X}")

        # 形状重排：experiments, echoes, slices, viewsSec, views, samples
        data = data.reshape(dims)

        return data, params

In [15]:
# Scan the directory and get an iterator of os.DirEntry objects
with os.scandir('Raw Data') as entries:
    for entry in entries:
        if entry.is_file() or entry.is_dir():
            print(entry.name)

cylinder
cylinder2
jhlefthandshielded
jhrighthandshielded
jhrighthandunshielded
noise1
noise2
noise3
shieldedPhantom
square


In [16]:
# Scan the directory and get an iterator of os.DirEntry objects
data_folder = "square" 

datetimecode_list = []
with os.scandir('Raw Data/'+data_folder) as entries:
    for entry in entries:
        if entry.is_file() or entry.is_dir():
            #print(entry.name)
            
            
            file_code = entry.name[0:-4]
            datetimecode_list.append(file_code)

In [17]:
print(len(datetimecode_list))

1014


In [ ]:
file_name = datetimecode_list[0]+".raw"


folder_path = "Raw Data\\"
#file_name = "M_board0_ch1_2025.9.13.14.7.49.592"
RAW_PATH = Path(folder_path+data_folder+'\\'+file_name)

#RAW_PATH = Path("shieldedPhantom\\M_board0_ch1_2025.9.17.16.40.42.870.raw")

# 固定偏移（基于手册）
OFF_NO_SAMPLES   = 0xFC00
OFF_NO_VIEWS     = 0xFC04
OFF_NO_VIEWSSEC  = 0xFC08
OFF_NO_SLICES    = 0xFC0C
OFF_DATATYPE     = 0xFC12  # 2 bytes
OFF_NO_ECHOES    = 0xFC98
OFF_NO_EXPS      = 0xFC9C

DATA_START       = 0x10108  # 真正数据起点

In [19]:
data, params = read_raw_firtech(RAW_PATH)
print("Params:", params)
# 典型：取第1个实验/回波/切片/副视角，得到 (noViews, noSamples)
kspace = data[0, 0, 0, 0, :, :]
print("kspace shape:", kspace.shape, "dtype:", kspace.dtype)
kspace = np.flipud(kspace)

Params: {'noSamples': 128, 'noViews': 128, 'noViewsSec': 1, 'noSlices': 1, 'noEchoes': 1, 'noExperiments': 1, 'dataTypeCode': 2}
kspace shape: (128, 128) dtype: complex64


In [ ]:
plt.imshow(abs(kspace.transpose()), cmap='gray')
plt.show()
image = abs(np.fft.ifftshift(np.fft.ifft2(kspace)))
plt.imshow(image, cmap='gray')
#plt.show()

folder_path = "Processed Data\\"
file_name = datetimecode_list[0]+".png"
plt.savefig(folder_path+data_folder+'\\'+file_name)

In [11]:
folder_path = "Processed Data\\"
file_name = datetimecode_list[0]+".npy"
#np.save(folder_path+data_folder+'\\'+file_name, image)

In [ ]:
loaded_array = np.load(folder_path+data_folder+'\\'+file_name)
print(loaded_array)
print("kspace shape:", loaded_array.shape, "dtype:", loaded_array.dtype)